In [ ]:
# Q1)

In [3]:
# import the modules
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 28 kB/s 
     |████████████████████████████████| 198 kB 54.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=cdc3afdfb62aa991d3c33c77b1345979a7aed0c383319ca2d4cac2d64e2a767e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
appName= "hive_pyspark"
master= "local"

In [ ]:
# Create spark session

In [5]:
spark = SparkSession.builder \
	.master(master).appName(appName).enableHiveSupport().getOrCreate()

In [6]:
df=spark.sql("show databases")
df.show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [20]:
# Q2)
df=spark.read.csv("boston.csv",header=True)
df.show(5)


+-----------+---+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM| ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+---+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632| 18|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|         24|
|0.027310001|  0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729|  0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001|  0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999|  0|2.18000006

In [22]:
df.printSchema()

root
 |-- CRIM: string (nullable = true)
 |-- ZN: string (nullable = true)
 |-- INDUS: string (nullable = true)
 |-- CHAS: string (nullable = true)
 |-- NOX: string (nullable = true)
 |-- RM: string (nullable = true)
 |-- AGE: string (nullable = true)
 |-- DIS: string (nullable = true)
 |-- RAD: string (nullable = true)
 |-- TAX: string (nullable = true)
 |-- PT: string (nullable = true)
 |-- B: string (nullable = true)
 |-- LSTAT: string (nullable = true)
 |-- MV: string (nullable = true)



In [24]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
CRIM,506,3.6135235608162057,8.601545086715594,0.00632,9.966540337
ZN,506,11.363636363636363,23.32245299451514,0,95
INDUS,506,11.136778749531626,6.86035298095724,0.460000008,9.899999619
CHAS,506,0.0691699604743083,0.2539940413404101,0,1
NOX,506,0.5546950602312246,0.1158776754570543,0.38499999,0.870999992
RM,506,6.28463438896641,0.7026171549511354,3.561000109,8.779999733
AGE,506,68.57490120115612,28.148861532793276,10,99.30000305
DIS,506,3.7950426960059325,2.105710142043288,1.129600048,9.222900391
RAD,506,9.549407114624506,8.707259384239366,1,8


In [16]:
# Q3) correlation
from pyspark.ml.stat import Correlation

IllegalArgumentException: ignored

In [17]:
datafile.stat.corr('CRIM','MV')

IllegalArgumentException: ignored

In [19]:
import six
for i in datafile.columns:
    if not( isinstance(datafile.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to MV for ", i, datafile.stat.corr('MV',i))

In [ ]:
# unable to print the correlation in colab notebook

In [ ]:
# Q4) Building LR model

In [30]:
# Changing the Data Type of the columns to double
from pyspark.sql.functions import col
df = df.select([col(column).cast('double') for column in df.columns])

In [32]:


from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')
v_df = vectorAssembler.transform(df)
v_df = v_df.select(['features', 'MV'])
v_df.show()

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
|[0.032370001,0.0,...|33.40000153|
|[0.069049999,0.0,...|36.20000076|
+--------------------+-----------+
only showing top 5 rows



In [33]:
# train test split
splits = v_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [35]:
# Linear Regression Model
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
#print("Coefficients: " + str(lr_model.coefficients))
#print("Intercept: " + str(lr_model.intercept))

In [36]:
# Q5)  Evaluating our model

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 4.759781
r2: 0.752972


In [37]:
# Therefore, R squared value of the Linear regression model is 0.7529
# and RMSE is 4.75